__Извлечение именованных сущностей из текстов__

Евгений Борисов <esborisov@sevsu.ru>

---

### извлечение имён

In [1]:
text = '''
Школа анализа данных возникла в 2007 году с целью восполнить 
недостаток квалифицированных кадров на рынке труда, с которым 
столкнулся Яндекс в то время. Научным руководителем ШАД стал 
профессор Ратгерского университета Илья Мучник, директором — Елена Бунина. 
'''

In [2]:
from ipymarkup import show_span_box_markup as show_markup
from natasha import MorphVocab
from natasha import NamesExtractor 

extrator = NamesExtractor(MorphVocab())

matches = list(  extrator(text) )
show_markup( text, [ (m.start,m.stop) for m in  matches ] )
display( [ dict( m.as_json['fact'].as_json ) for m in matches ]  )

[{'last': 'данных'},
 {'last': 'возникла'},
 {'last': 'в'},
 {'last': 'с'},
 {'last': 'целью'},
 {'last': 'кадров'},
 {'last': 'рынке'},
 {'last': 'труда'},
 {'last': 'с'},
 {'last': 'Яндекс'},
 {'last': 'в'},
 {'last': 'то'},
 {'last': 'ШАД'},
 {'last': 'стал'},
 {'last': 'Ратгерского'},
 {'first': 'Илья', 'last': 'Мучник'},
 {'last': 'директором'},
 {'first': 'Елена', 'last': 'Бунина'}]

### извлечение дат 

In [3]:
text = '24.01.2017, 2015 год, 2014 г, 1 апреля, май 2017 г., 9 мая 2017 года'

In [4]:
from ipymarkup import show_span_box_markup as show_markup
from natasha import MorphVocab
from natasha.extractors import DatesExtractor

extrator = DatesExtractor(MorphVocab())

matches = list(  extrator(text) )
show_markup( text, [ (m.start,m.stop) for m in  matches ] )
display( [ dict( m.as_json['fact'].as_json ) for m in matches ]  )

[{'year': 2017, 'month': 1, 'day': 24},
 {'year': 2015},
 {'year': 2014},
 {'month': 4, 'day': 1},
 {'year': 2017, 'month': 5},
 {'year': 2017, 'month': 5, 'day': 9}]

### извлечение адресов 

In [5]:
text = """Сегодня я поехал с аэропорта Толмачево в Академгородок -- 
мне нужно было привести посылку из Бостона, США. 
Мы ехали по Димитровскому мосту, потом свернули на улицу Большевицкую, 
ехали по ней. Остановились в кафе "хан-бууз", что находится по адресу Старое Шоссе 67А. 
Потом поехали на ул.Николаева 11, заехав по пути на Ляпунова 3"""

In [6]:
from ipymarkup import show_span_box_markup as show_markup
from natasha import MorphVocab
from natasha.extractors import AddrExtractor

extrator = AddrExtractor(MorphVocab())

matches = list(  extrator(text) )
show_markup( text, [ (m.start,m.stop) for m in  matches ] )
display( [ dict( m.as_json['fact'].as_json ) for m in matches ]  )

[{'value': 'Большевицкую', 'type': 'улица'},
 {'value': '67А', 'type': 'шоссе'},
 {'value': 'Николаева', 'type': 'улица'}]

### извлечение записей о денежных суммах 

In [7]:
text = '''Узнайте, сколько будет если сегодня и сейчас на дату 16.03.22 
совершить обмен суммы в размере 700 тысяч долларов(USD) 
в гривнах(UAH) по актуальному курсу, устанавливаемому ЦБ РФ. 
Любые другие суммы для пары USD-UAH можно рассчитать через конвертер.'''

In [8]:
from ipymarkup import show_span_box_markup as show_markup
from natasha import MorphVocab
from natasha.extractors import MoneyExtractor

extrator = MoneyExtractor(MorphVocab())

matches = list(  extrator(text) )
show_markup( text, [ (m.start,m.stop) for m in  matches ] )
display( [ dict( m.as_json['fact'].as_json ) for m in matches ]  )

[{'amount': 700000, 'currency': 'USD'}]

### используем сложную модель NER-таггера с Embedding

In [9]:
text = (
'В мероприятии примут участие не только российские учёные,'
' но и зарубежные исследователи, в том числе, Крис Хелмбрехт '
'- управляющий директор и совладелец креативного агентства'
' Kollektiv (Германия, США), Ннека Угбома - руководитель проекта'
' Mushroom works (Великобритания), Гергей Ковач - политик и лидер'
' субкультурной партии «Dog with two tails» (Венгрия),'
' Георг Жено - немецкий режиссёр, один из создателей экспериментального'
' театра «Театр.doc», Театра им. Йозефа Бойса (Германия).'
)

In [10]:
from ipymarkup import show_span_box_markup as show_markup
from natasha import Doc
from natasha import Segmenter
from natasha import NewsNERTagger
from natasha import NewsEmbedding

doc = Doc(text)
doc.segment(Segmenter())
doc.tag_ner( NewsNERTagger(NewsEmbedding()) )

show_markup(text, doc.ner.spans )
display( [ ( text[s.start:s.stop],s.type) for s in doc.ner.spans ] )

[('Крис Хелмбрехт', 'PER'),
 ('Kollektiv', 'ORG'),
 ('Германия', 'LOC'),
 ('США', 'LOC'),
 ('Ннека Угбома', 'PER'),
 ('Mushroom works', 'ORG'),
 ('Великобритания', 'LOC'),
 ('Гергей Ковач', 'PER'),
 ('Венгрия', 'LOC'),
 ('Георг Жено', 'PER'),
 ('Театр.doc', 'ORG'),
 ('Театра им.', 'ORG'),
 ('Йозефа Бойса', 'PER'),
 ('Германия', 'LOC')]